Ensure you have a benchmark, e.g. ./benchmark/ClinVarBenchmark_PB_202504.csv

In [1]:
!pip install biopython

In [2]:
import pandas as pd
import numpy as np
import subprocess
from tqdm import tqdm
import glob
from Bio import Entrez, SeqIO
import xml.etree.ElementTree as ET
import time

In [3]:
import os
import requests
import gzip
import shutil
import urllib.request
from bs4 import BeautifulSoup

# 1. Create directories
os.makedirs("benchmark", exist_ok=True)
os.makedirs("protein_dataset", exist_ok=True)

# 2. Download gene2accession.gz
gene2accession_url = "https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2accession.gz"
gene2accession_gz = "protein_dataset/gene2accession.gz"
gene2accession_file = "protein_dataset/gene2accession"

print("Downloading gene2accession.gz ...")
r = requests.get(gene2accession_url, stream=True)
with open(gene2accession_gz, "wb") as f:
    shutil.copyfileobj(r.raw, f)

# Unzip
print("Unzipping gene2accession.gz ...")
with gzip.open(gene2accession_gz, "rb") as f_in, open(gene2accession_file, "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)
os.remove(gene2accession_gz)

# 3. Scrape the FTP page for all human.*.protein.faa.gz files
base_url = "https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/mRNA_Prot/"
html_page = urllib.request.urlopen(base_url)
soup = BeautifulSoup(html_page, "html.parser")

faa_files = [a['href'] for a in soup.find_all('a', href=True) 
             if a['href'].startswith("human.") and a['href'].endswith(".protein.faa.gz")]

print(f"Found {len(faa_files)} protein FASTA files.")

# 4. Download and unzip each human.*.protein.faa.gz
for fname in faa_files:
    url = base_url + fname
    local_gz = os.path.join("protein_dataset", fname)
    local_faa = local_gz[:-3]

    print("Downloading:", fname)
    r = requests.get(url, stream=True)
    with open(local_gz, "wb") as f:
        shutil.copyfileobj(r.raw, f)

    print("Unzipping:", fname)
    with gzip.open(local_gz, "rb") as f_in, open(local_faa, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

    os.remove(local_gz)

print("✅ All files downloaded and unzipped into protein_dataset/")


KeyboardInterrupt



Change the input file path:

In [ ]:
Entrez.email = "anonymous.user@example.com"

# 1. Load your CSV and get unique NM IDs
df = pd.read_csv('./benchmark/ClinVarBenchmark_PB_202504.csv')

Preprocess actual protein sequences with thr refseq_id in benchmark

In [ ]:
gene2acc_path = './protein_dataset/gene2accession'

target_nm_ids = set(df['ClinVarName_refseq_ids'].dropna().unique())  # Only NM IDs we care about
print(f'Total targets = {len(target_nm_ids)}')

# 2. Build custom NM→NP mapping (first pass: gene2accession)
print("Building targeted NM→NP mapping (gene2accession)...")
nm_to_np = {}
with open(gene2acc_path) as f:
    total_lines = sum(1 for _ in f)

with open(gene2acc_path) as f:
    for line in tqdm(f, total=total_lines, desc="Scanning gene2accession"):
        if line.startswith('#'):
            continue
        parts = line.strip().split('\t')
        if len(parts) > 5:
            nm_id = parts[3]  # NM_XXXX.X
            np_id = parts[5]  # NP_XXXX.X
            if nm_id in target_nm_ids and np_id.startswith('NP_'):
                nm_to_np[nm_id] = np_id

# 3. Find missing NM IDs and fetch NP IDs via NCBI API
missing_nm_ids = target_nm_ids - set(nm_to_np.keys())
print(f"Found {len(missing_nm_ids)} unmapped NM IDs")

# 4. Fetch sequences for all NP IDs (from local FASTA or NCBI)
print("Fetching protein sequences...")

def get_sequence(np_id, fasta_files):
    """Search local FASTA files for NP sequence."""
    for fasta_file in fasta_files:
        with open(fasta_file) as f:
            record_id = None
            sequence = []
            for line in f:
                if line.startswith('>'):
                    if record_id == np_id:
                        return ''.join(sequence)
                    record_id = line[1:].split()[0]  # Get first word after >
                    sequence = []
                elif record_id == np_id:
                    sequence.append(line.strip())
            if record_id == np_id:
                return ''.join(sequence)
    return None

fasta_files = glob.glob('./protein_dataset/human.*.protein.faa')
nm_to_sequence = {}

# Try local FASTA files first
for nm_id, np_id in tqdm(nm_to_np.items(), desc="Processing sequences"):
    sequence = get_sequence(np_id, fasta_files)
    if sequence:
        nm_to_sequence[nm_id] = sequence
    else:
        missing_nm_ids.add(nm_id)

print(f"Found {len(missing_nm_ids)} NMs with no sequence.")


def fetch_protein_sequence_from_refseq(transcript_id):
    if transcript_id == 'NM_000557.1': # special case
        transcript_id = 'NM_000557.5'

    handle = Entrez.efetch(db="nucleotide", id=transcript_id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()

    for feature in record.features:
        if feature.type == "CDS" and "translation" in feature.qualifiers:
            protein_id = feature.qualifiers.get("protein_id", ["N/A"])[0]
            amino_acid_seq = feature.qualifiers["translation"][0]
            return protein_id, amino_acid_seq

    return None, None  

ext_nm_to_sequence = {}
for nm_id in tqdm(missing_nm_ids):
    protein_id, aa_seq = fetch_protein_sequence_from_refseq(nm_id)
    if aa_seq is not None:
        ext_nm_to_sequence[nm_id] = aa_seq

nm_to_sequence.update(ext_nm_to_sequence)

print(f"current NP seq = {len(nm_to_sequence)} / {len(target_nm_ids)}")
assert len(nm_to_sequence) == len(target_nm_ids)


# 5. Write FASTA (all successful mappings)
print("Writing FASTA output...")
with open('./protein_dataset/Preprocessed_all_prot.fasta', 'w') as f:
    for nm_id, seq in tqdm(nm_to_sequence.items()):
        f.write(f">{nm_id}\n{seq}\n")

print(f"Successfully mapped {len(nm_to_sequence)}/{len(target_nm_ids)} NM IDs")

Use https://github.com/ntranoslab/esm-variants to calculate ESM model scores (takes >10 GB disk space)

In [ ]:
!git clone https://github.com/ntranoslab/esm-variants
!python ./esm-variants/esm_score_missense_mutations.py --input-fasta-file protein_dataset/Preprocessed_all_prot.fasta --output-csv-file protein_dataset/ESM1b_score.csv --model-name esm1b_t33_650M_UR50S
!python ./esm-variants/esm_score_missense_mutations.py --input-fasta-file protein_dataset/Preprocessed_all_prot.fasta --output-csv-file protein_dataset/ESM1v_score.csv --model-name esm1v_t33_650M_UR90S_1
!python ./esm-variants/esm_score_missense_mutations.py --input-fasta-file protein_dataset/Preprocessed_all_prot.fasta --output-csv-file protein_dataset/ESM2_score.csv --model-name esm2_t33_650M_UR50D

Merge ESM model scores to 1 csv

In [ ]:
prefix = r'protein_dataset/'
reproduced_dict = {'esm1b_t33_650M_UR50S' : {'name' : 'ESM1b',
                                             'score_path' : prefix + 'ESM1b_score.csv'
                                            },
                   'esm1v_t33_650M_UR90S_1' : {'name' : 'ESM1v-1',
                                               'score_path' : prefix + 'ESM1v_score.csv'
                                              },
                   'esm2_t33_650M_UR50D' : {'name' : 'ESM2',
                                            'score_path' : prefix + 'ESM2_score.csv'
                                           }
                  }

output_path = r'./protein_dataset/ESM_score.csv'


merge_df = None
for model_pretrained, model_dict in reproduced_dict.items():
    score_df = pd.read_csv(model_dict['score_path'])

    if 'esm_score' in score_df.columns:
        score_df = score_df.rename(columns={'esm_score': model_dict['name'] + '_score'})

    if merge_df is None:
        merge_df = score_df
    else:
        merge_df = pd.merge(merge_df, score_df, on=['seq_id', 'mut_name'], how='inner')

# merge_df = reduce(lambda left, right: pd.merge(left, right, on=['seq_id', 'mut_name'], how='inner'), model_df_list)
merge_df.to_csv(output_path, index=False)

Calculate the stop_gain scores and filter out scores in the benchmark

In [ ]:
# Load data in chunks if needed
all_scores = pd.read_csv('./protein_dataset/ESM_score.csv')
benchmark = pd.read_csv('./benchmark/ClinVarBenchmark_PB_202504.csv')

# Preprocessing (unchanged)
benchmark = benchmark.rename(columns={'#CHROM': 'chrom', 'POS': 'pos', 'INFO': 'label'})
benchmark['chrom'] = benchmark['chrom'].astype(str).str.replace('chr', '')
benchmark['pos'] = benchmark['pos'].astype(int)
benchmark['ID'] = benchmark['ID'].astype(int)
# benchmark = benchmark[pd.to_numeric(benchmark['ClinVarName_AAPOS'], errors='coerce').notna()]
# benchmark['ClinVarName_AAPOS'] = benchmark['ClinVarName_AAPOS'].astype(int)

all_scores[['REF', 'POS', 'ALT']] = all_scores['mut_name'].str.extract(r'([A-Z*])(\d+)([A-Z*])')
all_scores['POS'] = all_scores['POS'].astype(int)

# Create merge keys and merge
benchmark['merge_key'] = (
    benchmark['ClinVarName_refseq_ids'].astype(str) + '_' +
    benchmark['ClinVarName_AAREF'].astype(str) +
    benchmark['ClinVarName_AAPOS'].apply(lambda x: str(int(x)) if pd.notna(x) else 'nan') +
    benchmark['ClinVarName_AAALT'].astype(str)
)

all_scores['merge_key'] = (
    all_scores['seq_id'].astype(str) + '_' +
    all_scores['REF'].astype(str) +
    all_scores['POS'].apply(lambda x: str(int(x)) if pd.notna(x) else 'nan') +
    all_scores['ALT'].astype(str)
)

models = ['ESM1b', 'ESM1v-1', 'ESM2']
result = benchmark.merge(
    all_scores[['merge_key'] + [f'{model}_score' for model in models]],
    on='merge_key',
    how='left'
).drop(columns = ['merge_key'])

# ========== PROPER DOWNSTREAM SCORING ==========
print("Finding true downstream worst scores...")

# Group variants by protein for efficient lookup
protein_variants = {
    seq_id: group[['POS'] + [f'{model}_score' for model in models]]
    for seq_id, group in all_scores.groupby('seq_id')
}

# Process stop codons in chunks
stop_mask = result['ClinVarName_AAALT'] == '*'
chunk_size = 5000  # Reduce if OOM occurs
n_chunks = (stop_mask.sum() // chunk_size) + 1

for i in tqdm(range(n_chunks), desc="Processing stop codons"):
    chunk = result[stop_mask].iloc[i*chunk_size : (i+1)*chunk_size]
    
    for idx, row in chunk.iterrows():
        refseq = row['ClinVarName_refseq_ids']
        stop_pos = row['ClinVarName_AAPOS']
        
        if refseq not in protein_variants:
            continue
            
        variants = protein_variants[refseq]
        downstream = variants[variants['POS'] >= stop_pos]
        
        if not downstream.empty:
            for model in models:
                worst_score = downstream[f'{model}_score'].min()
                result.at[idx, f'{model}_score'] = worst_score

result = result.rename(columns={'chrom' : '#CHROM', 'pos' : 'POS', 'label' : 'INFO'})
result = result.iloc[:, [0, 1, 2, -3, -2, -1] + list(range(3, result.shape[1] - 3))]

# Save results
result.to_csv(
    "./protein_dataset/benchmark_scores.csv", 
    index=False
)
print("Done")

fill in the benchmark

In [ ]:
DNA_scores = pd.read_csv("./benchmark/all_scores_v6.csv")
DNA_scores.drop(columns = ['ESM1b_score', 'ESM1v-1_score', 'ESM2_score'], inplace=True)
print(DNA_scores.columns.values)

protein_scores = pd.read_csv("./protein_dataset/benchmark_scores.csv")
protein_scores = protein_scores[['#CHROM', 'POS', 'ID', 'ESM1b_score', 'ESM1v-1_score', 'ESM2_score']]

# merge_cols = protein_scores.columns[[0, 1, 2] + list(range(6, protein_scores.shape[1]))].tolist()
merge_cols = protein_scores.columns[[0, 1, 2]].tolist()

all_scores = pd.merge(DNA_scores,
    protein_scores,
    on = merge_cols,
    how='inner'
)

print(all_scores.shape[0])
print('Any NA?')
print(all_scores.isna().any())

all_scores.to_csv('./benchmark/all_scores_final.csv', index = False)

print('model_merge.py done')